<a href="https://colab.research.google.com/github/Chirag314/Stacked-H2O-creditcarddata/blob/main/Stacked_H2O_creditcarddata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###This notebook is made from exercises from book Ensemble Machine Learning Cookbook.

H2O is an open source platform for building machine learning and predictive analytics models. The algorithms are written on H2O's distributed map-reduce framework. With H2O, the data is distributed across nodes, read in parallel, and stored in the memory in a compressed manner. This makes H2O extremely fast.

H2O's stacked ensemble method is an ensemble machine learning algorithm for supervised problems that finds the optimal combination of a collection of predictive algorithms using stacking. H2O's stacked ensemble supports regression, binary classification, and multiclass classification.
H2O's stacked ensemble method is an ensemble machine learning algorithm for supervised problems that finds the optimal combination of a collection of predictive algorithms using stacking. H2O's stacked ensemble supports regression, binary classification, and multiclass classification.

In [4]:
! apt-get install default-jre
! java -version

Reading package lists... Done
Building dependency tree       
Reading state information... Done
default-jre is already the newest version (2:1.11-68ubuntu1~18.04.1).
default-jre set to manually installed.
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 32 not upgraded.
openjdk version "11.0.16" 2022-07-19
OpenJDK Runtime Environment (build 11.0.16+8-post-Ubuntu-0ubuntu118.04)
OpenJDK 64-Bit Server VM (build 11.0.16+8-post-Ubuntu-0ubuntu118.04, mixed mode, sharing)


In [6]:
#Install H2O
! pip install h2o

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 177.1 MB 31 kB/s 
  Created wheel for h2o: filename=h2o-3.36.1.4-py2.py3-none-any.whl size=177128127 sha256=f120a79a80531873015387666277443948fd2a51b6ef293ac672c79cb9f23c8f
  Stored in directory: /root/.cache/pip/wheels/02/f9/18/5fbae4db903beda26f764b6e035cb41b37713fed8a6b9cd403
Successfully built h2o


In [8]:
#import required libraries

import h2o
from h2o.estimators.random_forest import H2ORandomForestEstimator
from h2o.estimators.gbm import H2OGradientBoostingEstimator
from h2o.estimators.glm import H2OGeneralizedLinearEstimator
from h2o.estimators.stackedensemble import H2OStackedEnsembleEstimator
from h2o.grid.grid_search import H2OGridSearch

In [9]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.16" 2022-07-19; OpenJDK Runtime Environment (build 11.0.16+8-post-Ubuntu-0ubuntu118.04); OpenJDK 64-Bit Server VM (build 11.0.16+8-post-Ubuntu-0ubuntu118.04, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.7/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpszilwkt8
  JVM stdout: /tmp/tmpszilwkt8/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpszilwkt8/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,03 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.36.1.4
H2O_cluster_version_age:,1 month and 9 days
H2O_cluster_name:,H2O_from_python_unknownUser_htpsew
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.172 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


In [23]:
# Read data from github. Use raw format and copy url# Note normal url and raw url will be different.
import pandas as pd
pd.options.display.max_rows=None
pd.options.display.max_columns=None
url = 'https://raw.githubusercontent.com/akhil12028/Bank-Marketing-data-set-analysis/master/bank-additional-full.csv'
hf_bankdata= h2o.import_file(url)
#df = pd.read_csv(url)
print(hf_bankdata.head(5))

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,261,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191,no
57,services,married,high.school,unknown,no,no,telephone,may,mon,149,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191,no
37,services,married,high.school,no,yes,no,telephone,may,mon,226,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191,no
40,admin.,married,basic.6y,no,no,no,telephone,may,mon,151,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191,no
56,services,married,high.school,no,no,yes,telephone,may,mon,307,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191,no


In [24]:
#Check shape of data
hf_bankdata.shape

(41188, 21)

In [25]:
#We first split the dataset into train and test subset
train,test=hf_bankdata.split_frame(ratios=[0.8],seed=1234)

In [26]:
train.shape, test.shape

((33054, 21), (8134, 21))

In [27]:
print(train.columns)

['age', 'job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'day_of_week', 'duration', 'campaign', 'pdays', 'previous', 'poutcome', 'emp.var.rate', 'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed', 'y']


In [28]:
# We separate the target and predictor column names, which are the response and predictors, respectively:
predictors=train.columns

#set the response column name
response="y"

#Remove the 'y variable from the predictor
predictors.remove(response)
print(predictors)

['age', 'job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'day_of_week', 'duration', 'campaign', 'pdays', 'previous', 'poutcome', 'emp.var.rate', 'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed']


In [30]:
#Convert response variable to a categorical type with the asfactor function
train[response]=train[response].asfactor()
test[response]=test[response].asfactor()


In [32]:
#We will train our base learners using cross-validation. We set the nfolds value to 5.We also set a variable 'encoding' to 'OneHotExplicit'. We will use this variable to encode our categorical variables.
#number of CV folds
nfolds=5

# Using the categorical_encoding parameter
encoding="OneHotExplicit"

In [33]:
#We start training our base learners. We choose the Gradient Boosting Machine algorithm to build our first base learner:
base_learner_gbm=H2OGradientBoostingEstimator(distribution='bernoulli',
                                              ntrees=100,
                                              max_depth=5,
                                              min_rows=2,
                                              learn_rate=0.01,
                                              nfolds=nfolds,
                                              fold_assignment="Modulo",
                                              categorical_encoding=encoding,
                                              keep_cross_validation_predictions=True)
base_learner_gbm.train(x=predictors,y=response,training_frame=train)

gbm Model Build progress: |██████████████████████████████████████████████████████| (done) 100%
Model Details
H2OGradientBoostingEstimator :  Gradient Boosting Machine
Model Key:  GBM_model_python_1663050917904_1


Model Summary: 


,,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
0,,100.0,100.0,46011.0,5.0,5.0,5.0,31.0,32.0,31.94




ModelMetricsBinomial: gbm
** Reported on train data. **

MSE: 0.06138054125314093
RMSE: 0.24775096620021672
LogLoss: 0.21098084712555518
Mean Per-Class Error: 0.1746786630743241
AUC: 0.9440474206801891
AUCPR: 0.6918704113541194
Gini: 0.8880948413603782

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.27398031300288583: 


,,no,yes,Error,Rate
0,no,27676.0,1654.0,0.0564,(1654.0/29330.0)
1,yes,1091.0,2633.0,0.293,(1091.0/3724.0)
2,Total,28767.0,4287.0,0.083,(2745.0/33054.0)



Maximum Metrics: Maximum metrics at their respective thresholds


...


Scoring History: 


,,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_auc,training_pr_auc,training_lift,training_classification_error
0,,2022-09-13 07:02:11,1 min 33.881 sec,0.0,0.316182,0.352049,0.500000,0.112664,1.000000,0.887336
1,,2022-09-13 07:02:12,1 min 35.000 sec,1.0,0.314792,0.347737,0.931339,0.661420,7.418696,0.096963
2,,2022-09-13 07:02:12,1 min 35.233 sec,2.0,0.313426,0.343649,0.931370,0.661998,7.418696,0.096963
3,,2022-09-13 07:02:12,1 min 35.473 sec,3.0,0.312103,0.339813,0.932816,0.665798,7.422700,0.091033
4,,2022-09-13 07:02:13,1 min 35.780 sec,4.0,0.310803,0.336156,0.932674,0.658100,7.332298,0.087856
5,,2022-09-13 07:02:13,1 min 36.080 sec,5.0,0.309503,0.332604,0.935059,0.666953,7.422700,0.091033
6,,2022-09-13 07:02:13,1 min 36.332 sec,6.0,0.308245,0.329252,0.934948,0.659475,7.340185,0.087856
7,,2022-09-13 07:02:14,1 min 36.572 sec,7.0,0.307008,0.326037,0.935011,0.660939,7.409593,0.086676
8,,2022-09-13 07:02:14,1 min 36.877 sec,8.0,0.305769,0.322897,0.934971,0.660849,7.409593,0.087856
9,,2022-09-13 07:02:14,1 min 37.128 sec,9.0,0.304572,0.319922,0.935042,0.661087,7.409593,0.086676



Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,duration,31855.388672,1.000000,0.503293
1,nr.employed,22311.763672,0.700408,0.352510
2,euribor3m,1772.733521,0.055649,0.028008
3,cons.conf.idx,1724.984497,0.054150,0.027254
4,pdays,1503.662354,0.047203,0.023757
5,cons.price.idx,925.715027,0.029060,0.014626
6,poutcome.success,922.239441,0.028951,0.014571
7,month.oct,738.470764,0.023182,0.011667
8,contact.cellular,321.591980,0.010095,0.005081
9,month.mar,230.650970,0.007241,0.003644



See the whole table with table.as_data_frame()


In [35]:
#For our second base learner, we use a Random Forest:
#Train and cross-validate a RF
base_learner_rf=H2ORandomForestEstimator(ntrees=250,
                                         nfolds=nfolds,
                                         fold_assignment="Modulo",
                                         categorical_encoding=encoding,
                                         keep_cross_validation_predictions=True)
base_learner_rf.train(x=predictors,y=response,training_frame=train)

drf Model Build progress: |██████████████████████████████████████████████████████| (done) 100%
Model Details
H2ORandomForestEstimator :  Distributed Random Forest
Model Key:  DRF_model_python_1663050917904_175


Model Summary: 


,,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
0,,250.0,250.0,6843079.0,20.0,20.0,20.0,1729.0,2448.0,2163.272




ModelMetricsBinomial: drf
** Reported on train data. **

MSE: 0.05751872630254877
RMSE: 0.23983062002702818
LogLoss: 0.18102744375131438
Mean Per-Class Error: 0.15795183919567074
AUC: 0.942829122694711
AUCPR: 0.6480747987069494
Gini: 0.885658245389422

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.3115585924264776: 


,,no,yes,Error,Rate
0,no,27019.0,2311.0,0.0788,(2311.0/29330.0)
1,yes,883.0,2841.0,0.2371,(883.0/3724.0)
2,Total,27902.0,5152.0,0.0966,(3194.0/33054.0)



Maximum Metrics: Maximum metrics at their respective thresholds


...


Scoring History: 


,,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_auc,training_pr_auc,training_lift,training_classification_error
0,,2022-09-13 07:12:36,8 min 23.207 sec,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1,,2022-09-13 07:12:36,8 min 23.473 sec,1.0,0.321015,3.151144,0.762383,0.392571,4.712205,0.119261
2,,2022-09-13 07:12:37,8 min 23.871 sec,2.0,0.316980,2.834902,0.762004,0.385411,4.642977,0.123658
3,,2022-09-13 07:12:37,8 min 24.189 sec,3.0,0.308186,2.500635,0.774444,0.401877,4.830942,0.122728
4,,2022-09-13 07:12:37,8 min 24.587 sec,4.0,0.301191,2.222025,0.792173,0.418761,4.919509,0.123632
5,,2022-09-13 07:12:38,8 min 24.856 sec,5.0,0.292838,1.885032,0.811173,0.440420,5.029699,0.124605
6,,2022-09-13 07:12:38,8 min 25.125 sec,6.0,0.287369,1.654489,0.825168,0.456870,5.132997,0.119138
7,,2022-09-13 07:12:38,8 min 25.404 sec,7.0,0.281912,1.424767,0.838666,0.471775,5.266391,0.119476
8,,2022-09-13 07:12:38,8 min 25.667 sec,8.0,0.277506,1.220311,0.850649,0.484441,5.344467,0.118401
9,,2022-09-13 07:12:39,8 min 25.957 sec,9.0,0.274211,1.068119,0.860772,0.494116,5.428095,0.116107



See the whole table with table.as_data_frame()

Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,duration,138559.609375,1.000000,0.283195
1,euribor3m,42926.558594,0.309806,0.087735
2,age,34140.230469,0.246394,0.069777
3,nr.employed,29592.912109,0.213575,0.060483
4,pdays,17697.777344,0.127727,0.036172
5,campaign,15173.523438,0.109509,0.031012
6,emp.var.rate,14040.378906,0.101331,0.028696
7,cons.conf.idx,13668.318359,0.098646,0.027936
8,poutcome.success,12860.922852,0.092819,0.026286
9,cons.price.idx,12365.352539,0.089242,0.025273



See the whole table with table.as_data_frame()


In [36]:
# For our third base learner, we implement a Generalized Linear Model (GLM):
base_learner_glm=H2OGeneralizedLinearEstimator(family="binomial",
                                               model_id='GLM',
                                               lambda_search=True,
                                               nfolds=nfolds,
                                               fold_assignment="Modulo",
                                               keep_cross_validation_predictions=True)

base_learner_glm.train(x=predictors,y=response,training_frame=train)

glm Model Build progress: |██████████████████████████████████████████████████████| (done) 100%
Model Details
H2OGeneralizedLinearEstimator :  Generalized Linear Modeling
Model Key:  GLM


GLM Model: summary


,,family,link,regularization,lambda_search,number_of_predictors_total,number_of_active_predictors,number_of_iterations,training_frame
0,,binomial,logit,"Elastic Net (alpha = 0.5, lambda = 4.898E-5 )","nlambda = 100, lambda.max = 0.2554, lambda.min = 4.898E-5, lambda....",63,48,133,py_13_sid_b51b




ModelMetricsBinomialGLM: glm
** Reported on train data. **

MSE: 0.062150544449254386
RMSE: 0.24930010920425683
LogLoss: 0.20676677496156093
Null degrees of freedom: 33053
Residual degrees of freedom: 33005
Null deviance: 23273.277611826466
Residual deviance: 13668.937959158871
AIC: 13766.937959158871
AUC: 0.9372907254132116
AUCPR: 0.6031869451809622
Gini: 0.8745814508264231

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.2534388791869134: 


,,no,yes,Error,Rate
0,no,27300.0,2030.0,0.0692,(2030.0/29330.0)
1,yes,1115.0,2609.0,0.2994,(1115.0/3724.0)
2,Total,28415.0,4639.0,0.0951,(3145.0/33054.0)



Maximum Metrics: Maximum metrics at their respective thresholds


...


See the whole table with table.as_data_frame()

Scoring History: 


,,timestamp,duration,iteration,lambda,predictors,deviance_train,deviance_xval,deviance_se,alpha,iterations,training_rmse,training_logloss,training_r2,training_auc,training_pr_auc,training_lift,training_classification_error
0,,2022-09-13 07:14:26,0.000 sec,1,.26E0,1,0.704099,0.704140,0.005562,0.5,NaN,,,,,,,
1,,2022-09-13 07:14:26,0.053 sec,3,.23E0,2,0.691394,0.704099,0.005575,0.5,NaN,,,,,,,
2,,2022-09-13 07:14:26,0.098 sec,5,.21E0,3,0.673779,0.704099,0.005575,0.5,NaN,,,,,,,
3,,2022-09-13 07:14:26,0.130 sec,7,.19E0,4,0.651783,0.696350,0.005485,0.5,NaN,,,,,,,
4,,2022-09-13 07:14:26,0.155 sec,9,.18E0,4,0.629303,0.682412,0.006033,0.5,NaN,,,,,,,
5,,2022-09-13 07:14:26,0.178 sec,11,.16E0,4,0.609576,0.661195,0.005855,0.5,NaN,,,,,,,
6,,2022-09-13 07:14:26,0.204 sec,13,.15E0,5,0.591613,0.638167,0.005791,0.5,NaN,,,,,,,
7,,2022-09-13 07:14:26,0.231 sec,15,.13E0,6,0.575311,0.617394,0.005569,0.5,NaN,,,,,,,
8,,2022-09-13 07:14:26,0.256 sec,17,.12E0,6,0.560710,0.598876,0.005435,0.5,NaN,,,,,,,
9,,2022-09-13 07:14:26,0.293 sec,19,.11E0,6,0.547781,0.581863,0.005285,0.5,NaN,,,,,,,



See the whole table with table.as_data_frame()

Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,emp.var.rate,2.389294,1.000000,0.167407
1,month.mar,1.741694,0.728957,0.122032
2,duration,1.226243,0.513224,0.085917
3,cons.price.idx,1.060092,0.443684,0.074276
4,month.may,0.670142,0.280477,0.046954
5,month.aug,0.621700,0.260203,0.043560
6,euribor3m,0.591005,0.247355,0.041409
7,month.nov,0.579589,0.242577,0.040609
8,month.jun,0.571780,0.239309,0.040062
9,poutcome.success,0.525921,0.220116,0.036849



See the whole table with table.as_data_frame()


In [37]:
# Get the best-performing base learner on the test set in terms of the test AUC. Compare this with the test AUC of the stacked ensemble model:
gbm_test_performance=base_learner_gbm.model_performance(test)
rf_test_performance=base_learner_rf.model_performance(test)
glm_test_performance=base_learner_glm.model_performance(test)

print("Best AUC from the GBM ", gbm_test_performance.auc())
print("Best AUC from the Random Forest", rf_test_performance.auc())
print("Best AUC from the GLM ", glm_test_performance.auc())

base_learner_best_auc_test=max(gbm_test_performance.auc(),rf_test_performance.auc(),glm_test_performance.auc())
print("Best AUC from the base learners", base_learner_best_auc_test)




Best AUC from the GBM  0.9307436315809215
Best AUC from the Random Forest 0.93803034867949
Best AUC from the GLM  0.9302674294370817
Best AUC from the base learners 0.93803034867949


We train a stacked ensemble using the base learners we built in the preceding step

In [41]:
all_models=[base_learner_glm,base_learner_gbm,base_learner_rf]
#Set up stacked ensemble using deep learning as meta ensemble
ensemble_deep=H2OStackedEnsembleEstimator(model_id="stack_model_d",base_models=all_models,metalearner_algorithm='deeplearning')
ensemble_deep.train(y=response,training_frame=train)

#Eval ensemble performance on the test data
perf_stack_test=ensemble_deep.model_performance(test)
stack_auc_test=perf_stack_test.auc()
print("ensemble_deep test AUC; {0}".format(stack_auc_test))


stackedensemble Model Build progress: |██████████████████████████████████████████| (done) 100%
ensemble_deep test AUC; 0.9411725114675707


You may also assemble a list of models to stack together in different ways. In the preceding example, we trained individual models and put them in a list to ensemble them. We can also train a grid of models:

We specify the random forest hyperparameters for the grid:


In [50]:
hyper_params={"max_depth":[3,4,5,8,10],
              "min_rows":[3,4,5,6,7,8,9,10],
              "mtries":[10,15,20],
              'ntrees':[100,250,500,750],
              "sample_rate":[0.7,0.8,0.9,1.0],
              "col_sample_rate_per_tree":[0.5,0.6,0.7,0.8,0.9,1.0]}
search_criteria={"strategy":"RandomDiscrete","max_models":3, "seed":1}


In [51]:
#We train the grid using the hyperparameters defined in the preceding code:
grid=H2OGridSearch(model=H2ORandomForestEstimator(nfolds=nfolds,
                                                  keep_cross_validation_predictions=True),
                   hyper_params=hyper_params,
                   search_criteria=search_criteria,
                   grid_id='rf_grid_binomial'
                   )
grid.train(x=predictors,y=response,training_frame=train)

drf Grid Build progress: |███████████████████████████████████████████████████████| (done) 100%
     col_sample_rate_per_tree  max_depth  min_rows  mtries  ntrees  \
0                         0.8        8.0       6.0    10.0   100.0   
1                         0.6        8.0      10.0    15.0   750.0   
2                         0.7        4.0      10.0    15.0   100.0   

   sample_rate                 model_ids   logloss  
0          0.7  rf_grid_binomial_model_1  0.179610  
1          1.0  rf_grid_binomial_model_3  0.192846  
2          0.7  rf_grid_binomial_model_2  0.199156  


In [53]:
# Train a stacked ensemble using the RF grid
ensemble = H2OStackedEnsembleEstimator(model_id="ensemble_rf_grid_binomial_9", base_models=grid.model_ids)

ensemble.train(x=predictors, y=response, training_frame=train)

# Evaluate ensemble performance on the test data
perf_stack_test = ensemble.model_performance(test)

# Compare to base learner performance on the test set
baselearner_best_auc_test = max([h2o.get_model(model).model_performance(test_data=test).auc() for model in grid.model_ids])

stack_auc_test = perf_stack_test.auc()

print("Best Base-learner Test AUC: ", baselearner_best_auc_test)
print("Ensemble Test AUC: ", stack_auc_test)

stackedensemble Model Build progress: | (failed)


OSError: ignored

The preceding code will give the best base-learner test AUC and test the AUC from the ensemble model. If the response variable is highly imbalanced, consider fine-tuning the following hyperparameters to control oversampling and under-sampling:

balance_classes: This option can be used to balance the class distribution. When enabled, H2O will either under-sample the majority classes or oversample the minority classes. If this option is enabled, you can also specify a value for the class_sampling_factors and max_after_balance_size options.
class_sampling_factors: By default, sampling factors will be automatically computed to obtain class balance during training. This behavior may be changed using the class_sampling_factors parameter. This option sets an over- or under-sampling ratio for each class and requires balance_classes=true.
max_after_balance_size: In most cases, setting balance_classes to true will increase the size of the DataFrame. To reduce the DataFrame size, you can use the max_after_balance_size parameter. This specifies the maximum relative size of the training data after balancing the class counts and defaults to 5.0.